使用PyTorch框架搭建一个简单的SNN模型。
脉冲神经网络（SNN）是一种模拟生物神经元行为的神经网络模型，具有较高的计算效率和能量效率

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

定义一个脉冲神经元（spiking neuron）类

In [2]:
class SpikingNeuron(nn.Module):
    def __init__(self, threshold=1.0, decay=0.9):
        super(SpikingNeuron, self).__init__()
        self.threshold = threshold
        self.decay = decay
        self.membrane_potential = 0

    def forward(self, x):
        self.membrane_potential += x
        spike = (self.membrane_potential >= self.threshold).float()
        self.membrane_potential = self.membrane_potential * (1 - spike) * self.decay
        return spike

    

In [3]:
# 定义一个简单的SNN模型，包含一个输入层、一个隐藏层和一个输出层：

class SNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SNN, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.hidden_layer = SpikingNeuron()
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.input_layer(x)
        x = self.hidden_layer(x)
        x = self.output_layer(x)
        return x

    

In [4]:
# 生成数据样例
X = torch.randn(1000, 2)
y = (X[:, 0] + X[:, 1] > 0).float()

# 创建数据加载器
dataset = data.TensorDataset(X, y)
data_loader = data.DataLoader(dataset, batch_size=10, shuffle=True)

    


model = SNN(input_size=2, hidden_size=10, output_size=1)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

  

In [6]:
num_epochs = 200

for epoch in range(num_epochs):
    epoch_loss = 0
    correct = 0
    total = 0

    for X_batch, y_batch in data_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs.view(-1), y_batch)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        correct += ((outputs.view(-1) > 0) == y_batch).sum().item()
        total += y_batch.size(0)

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / total:.4f}, Accuracy: {correct / total:.4f}')

    


# 生成测试数据
X_test = torch.randn(10, 2)
y_test = (X_test[:, 0] + X_test[:, 1] > 0).float()

# 测试模型
with torch.no_grad():
    outputs = model(X_test)
    test_loss = criterion(outputs.view(-1), y_test)
    test_accuracy = ((outputs.view(-1) > 0) == y_test).sum().item() / y_test.size(0)

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')



Epoch 1/200, Loss: 0.0639, Accuracy: 0.6280
Epoch 2/200, Loss: 0.0566, Accuracy: 0.7680
Epoch 3/200, Loss: 0.0540, Accuracy: 0.7610
Epoch 4/200, Loss: 0.0519, Accuracy: 0.7590
Epoch 5/200, Loss: 0.0514, Accuracy: 0.7740
Epoch 6/200, Loss: 0.0505, Accuracy: 0.7720
Epoch 7/200, Loss: 0.0499, Accuracy: 0.7780
Epoch 8/200, Loss: 0.0514, Accuracy: 0.7620
Epoch 9/200, Loss: 0.0509, Accuracy: 0.7740
Epoch 10/200, Loss: 0.0495, Accuracy: 0.7720
Epoch 11/200, Loss: 0.0505, Accuracy: 0.7680
Epoch 12/200, Loss: 0.0523, Accuracy: 0.7520
Epoch 13/200, Loss: 0.0503, Accuracy: 0.7700
Epoch 14/200, Loss: 0.0472, Accuracy: 0.7990
Epoch 15/200, Loss: 0.0488, Accuracy: 0.7780
Epoch 16/200, Loss: 0.0486, Accuracy: 0.7830
Epoch 17/200, Loss: 0.0486, Accuracy: 0.7870
Epoch 18/200, Loss: 0.0508, Accuracy: 0.7650
Epoch 19/200, Loss: 0.0514, Accuracy: 0.7610
Epoch 20/200, Loss: 0.0475, Accuracy: 0.7940
Epoch 21/200, Loss: 0.0495, Accuracy: 0.7680
Epoch 22/200, Loss: 0.0498, Accuracy: 0.7770
Epoch 23/200, Loss: